## Importing Libraries

In [0]:
import pandas as pd, numpy as np
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"

In [0]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [0]:
L1_preds_w_PIM = pd.read_csv(DBFR + 'L1_Preds_1_1Mn_data.csv')
# L1_preds_w_PIM = pd.read_csv(DBFR + 'L1_Predictionz__Synd_Part_12345.csv')

synd_all = pd.read_csv(DBFR + "Syndigo_Final_ALL.csv")
pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object) 

In [0]:
if 'GTIN_NO' in pim_gtin_mapped.columns.tolist():
    pim_gtin_mapped.rename(columns={'GTIN_NO':'GTIN'}, inplace=True)
if 'Unnamed: 0.1' in pim_gtin_mapped.columns.tolist():
    pim_gtin_mapped.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD']:
    pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)

pim_gtin_mapped['ITM_ID'] = pim_gtin_mapped['ITM_ID'].astype(np.float64)
pim_gtin_mapped['GTIN'] = pim_gtin_mapped['GTIN'].astype(np.float64)
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD']:
    pim_gtin_mapped[str(i)] = pim_gtin_mapped[str(i)].astype(np.float64)
    # pim_gtin_mapped[str(i)] = pim_gtin_mapped[str(i)].fillna(0)
    # pim_gtin_mapped[str(i)] = pim_gtin_mapped[str(i)].astype(np.int64)
print(pim_gtin_mapped.shape)
print(pim_gtin_mapped.columns)

(5082212, 23)
Index(['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
       'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
       'SUBCOM_DSC', 'VND_ECOM_DSC', 'Level 1', 'Level 2', 'Level 3',
       'Level 4', 'Level 5', 'Level 6', 'Level 7', 'Level 8', 'Level 9',
       'Level 10'],
      dtype='object')


In [0]:
import pandas as pd
L2_Preds = pd.read_csv(DBFR + 'L1_L2_FINAL_synd.csv')
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'L1 Scores', 'L2 Scores', 'PMY_DPT_CD']:
    L2_Preds[i] = L2_Preds[i].astype(np.float64)
L2_Preds.columns

if 'Unnamed: 0' in pim_gtin_mapped.columns.tolist():
    pim_gtin_mapped.drop(columns=['Unnamed: 0'], inplace=True)

## Level 2 Classifier

In [0]:
L1_preds_w_PIM.sort_values(by=['DPT_CD', 'VND_ECOM_DSC', 'First Scores'], ascending=[True, True, False], inplace=True)
L1_preds_w_PIM.drop_duplicates(subset=['DPT_CD', 'VND_ECOM_DSC'], inplace=True)

In [0]:
L1_preds_w_PIM.rename(columns={'First L1 Predictions':'L1 Predictions', 'First Scores':'Scores'}, inplace=True)
L1_preds_w_PIM.drop(columns=['Second L1 Predictions', 'Second Scores', 'Third L1 Predictions','Third Scores'], inplace=True)

In [0]:
l2_raw_data = pd.concat([L1_preds_w_PIM, synd_all], ignore_index=True)
l2_raw_data['ITEM_SUBCOM_text'] = (l2_raw_data.VND_ECOM_DSC + ' ' + l2_raw_data.DPT_DSC).fillna('').str.lower()

In [0]:
print(len(L1_preds_w_PIM)+len(synd_all)) # 3046898

801214


In [0]:
l2_raw_data[l2_raw_data['PMY_DPT_CD'].notnull()] # 3044615
len(l2_raw_data[l2_raw_data['PMY_DPT_CD'].isna()]) # 2283
# 3046898

In [0]:
l2_raw_data['Level 1'].replace('', np.nan, inplace=True)
l2_raw_data['Level 2'].replace('', np.nan, inplace=True)
l2_raw_data['Level 3'].replace('', np.nan, inplace=True)
l2_raw_data['Level 4'].replace('', np.nan, inplace=True)
l2_raw_data['Level 5'].replace('', np.nan, inplace=True)
l2_raw_data['Level 6'].replace('', np.nan, inplace=True)
l2_raw_data['Level 7'].replace('', np.nan, inplace=True)
l2_raw_data['Level 8'].replace('', np.nan, inplace=True)
l2_raw_data['Level 9'].replace('', np.nan, inplace=True)
l2_raw_data['Level 10'].replace('', np.nan, inplace=True)

## CHECKPOINT

In [0]:
for i in l2_raw_data[l2_raw_data['L1 Predictions'].notnull()]['L1 Predictions'].unique().tolist():
    print(f"level_1 = \"{i}\"")
    # print(i.replace('/', '_').replace('-', '_').replace(' ', '') + '_L2_Synd_ALL.csv' + ',')

level_1 = "Food / Beverages"
level_1 = "Health & Beauty"
level_1 = "Furniture"
level_1 = "Gardening & Outdoors"
level_1 = "Beer / Wine / Spirits"
level_1 = "Home & Venue Decoration"
level_1 = "Arts & Crafts"
level_1 = "Office Supplies"
level_1 = "Apparel"
level_1 = "Books & Videos"
level_1 = "Livestock & Pet Supplies"
level_1 = "Heating / Ventilation / Air Conditioning"
level_1 = "Toys / Games / Hobbies"
level_1 = "Appliances"
level_1 = "Electronics"
level_1 = "Kitchen & Bathroom"
level_1 = "Cleaning & Janitorial"
level_1 = "Lighting & Fans"
level_1 = "Hardware"
level_1 = "Automotive"
level_1 = "Childcare"
level_1 = "Hospitality Supplies"
level_1 = "Sports & Outdoor Recreation Equipment"
level_1 = "Tools"
level_1 = "Tobacco Products"
level_1 = "Electrical"
level_1 = "Plumbing & Water Service"
level_1 = "Marine"
level_1 = "Power Sports"
level_1 = "Building Supplies"
level_1 = "Flooring"
level_1 = "Paints & Coatings"
level_1 = "Material Handling"


In [0]:
print("\"RUN ALL BELOW\"")

In [0]:
# level_1 = "Livestock & Pet Supplies"
# level_1 = "Food / Beverages"
# level_1 = "Health & Beauty"
# level_1 = "Furniture"
# level_1 = "Gardening & Outdoors"
# level_1 = "Beer / Wine / Spirits"
# level_1 = "Home & Venue Decoration"
# level_1 = "Arts & Crafts"
# level_1 = "Office Supplies"
# level_1 = "Apparel"
# level_1 = "Books & Videos"
# level_1 = "Heating / Ventilation / Air Conditioning"
# level_1 = "Toys / Games / Hobbies"
# level_1 = "Appliances"
# level_1 = "Electronics"
# level_1 = "Kitchen & Bathroom"
# level_1 = "Cleaning & Janitorial"
# level_1 = "Lighting & Fans"
# level_1 = "Hardware"
# level_1 = "Automotive"
# level_1 = "Childcare"
# level_1 = "Hospitality Supplies"
# level_1 = "Sports & Outdoor Recreation Equipment"
# level_1 = "Tools"
# level_1 = "Tobacco Products"
# level_1 = "Electrical"
# level_1 = "Plumbing & Water Service"
# level_1 = "Marine" # 3
# level_1 = "Power Sports"
# level_1 = "Building Supplies"
# level_1 = "Flooring"
# level_1 = "Paints & Coatings"
level_1 = "Material Handling"

In [0]:
l1_filter = pd.concat([l2_raw_data[l2_raw_data['Level 1']==level_1], l2_raw_data[l2_raw_data['L1 Predictions']==level_1]], ignore_index=True)
print("L1 Filter                   = ",len(l1_filter))
# Sorting

l1_filter = l1_filter.sort_values(by=['Level 1'])
print("Level 1 NotNull             = " ,len(l1_filter[l1_filter['Level 1'].notnull()]))
print("L1 Filter - Level 1 NotNull = ", len(l1_filter) - len(l1_filter[l1_filter['Level 1'].notnull()]))
print("Level 1 Predictions         = ", len(l1_filter[l1_filter['L1 Predictions'].notnull()]))

L1 Filter                   =  10
Level 1 NotNull             =  6
L1 Filter - Level 1 NotNull =  4
Level 1 Predictions         =  4


In [0]:
# Creating a copy
subset_df = l1_filter
print(len(subset_df))

vect_subset = TfidfVectorizer(max_features = 8000000)
# X_subset = vect_subset.fit_transform((subset_df.VND_ECOM_DSC + ' ' + subset_df.SUBCOM_DSC).fillna('').str.lower())
X_subset = vect_subset.transform(subset_df['ITEM_SUBCOM_text'])

(X_subset.shape)

10
Out[462]: (10, 65)

In [0]:
l2_only = X_subset[:len(l1_filter[l1_filter['Level 1'].notnull()])][:] #X_train
non_l2 = X_subset[len(l1_filter[l1_filter['Level 1'].notnull()]):][:]  #X_test
print(l2_only.shape)
print(non_l2.shape)

(6, 65)
(4, 65)


In [0]:
# Training
subset_df = l1_filter[l1_filter['Level 1'].notnull()]
level2_id_map = dict(zip(subset_df['Level 2'].fillna('').unique(), range(subset_df['Level 2'].fillna('').nunique())))
y_subset  = subset_df['Level 2'].fillna('').map(level2_id_map)
lr_tf = LogisticRegression(C = 1.0, multi_class = 'multinomial', solver = 'saga', n_jobs=64)
# lr_tf.fit(X_train, y_train)
lr_tf.fit(l2_only, y_subset)

# Predict
# preds = lr_tf.predict(X_test)
preds = lr_tf.predict(non_l2)
probs = lr_tf.predict_proba(non_l2)
print()
print(len(preds))
print(len(y_subset))


4
6


/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [0]:
print(len(preds))
print(len(probs))
len(probs[0])

4
4
Out[465]: 4

In [0]:
level_num = []
proobs = []
number = 3

for i in range(len(probs)): # 606554
    for j in range(number):
        try:
            level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][j])[0]))
        except TypeError:
            level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][2])[0][0]))
    proobs.append(probs[i][np.argsort(probs[i])][::-1][:3])

soln = []
for k in range(len(level_num)):
    soln.append([i for i in level2_id_map if level2_id_map[i]==level_num[k]][0])

In [0]:
print(len(proobs))
print(len(soln))
print(len(level_num))
print(len(proobs)*3)
proobs[0]

4
12
12
12
Out[467]: array([0.39882048, 0.20789401, 0.19943159])

In [0]:
new_proobs = []
for i in range(len(proobs)):
    new_proobs.append(proobs[i].tolist())
new_proobs = [element for sublist in list(new_proobs) for element in sublist]
print(len(soln))
print(len(new_proobs))

12
12


In [0]:
l2_preds = l1_filter[l1_filter['L1 Predictions']==level_1]
datas = {
'PMY_DPT_CD' : [ele for ele in l2_preds.PMY_DPT_CD.tolist() for i in range(number)],
'PMY_DPT_DSC': [ele for ele in l2_preds.PMY_DPT_DSC.tolist() for i in range(number)],
'REC_DPT_CD' : [ele for ele in l2_preds.REC_DPT_CD.tolist() for i in range(number)],
'REC_DPT_DSC' : [ele for ele in l2_preds.REC_DPT_DSC.tolist() for i in range(number)],
'DPT_CD' : [ele for ele in l2_preds.DPT_CD.tolist() for i in range(number)], 
'DPT_DSC' : [ele for ele in l2_preds.DPT_DSC.tolist() for i in range(number)],
'COM_CD' : [ele for ele in l2_preds.COM_CD.tolist() for i in range(number)],
'COM_DSC' : [ele for ele in l2_preds.COM_DSC.tolist() for i in range(number)],
'SUBCOM_CD' : [ele for ele in l2_preds.SUBCOM_CD.tolist() for i in range(number)],
'SUBCOM_DSC' : [ele for ele in l2_preds.SUBCOM_DSC.tolist() for i in range(number)],
'VND_ECOM_DSC' : [ele for ele in l2_preds.VND_ECOM_DSC.tolist() for i in range(number)],
'ITEM_SUBCOM_text' : [ele for ele in l2_preds.ITEM_SUBCOM_text.tolist() for i in range(number)],
'L1 Predictions' : [ele for ele in l2_preds['L1 Predictions'].tolist() for i in range(number)],
'L1 Scores' : [ele for ele in l2_preds['Scores'].tolist() for i in range(number)],

'L2 Predictions' : soln,
'L2 Scores' : new_proobs
}

L2_Preds = pd.DataFrame(datas)

print(len(L2_Preds))

12
